In [1]:
import os
import numpy
import torch
import matplotlib.pyplot as plt
import sklearn
import pickle
import pandas
import gzip
import sys
import plotly_express as px
import plotly.graph_objects as go
import functools
sys.path.insert(0, os.path.abspath('../../'))
from marrovision.cortex.data.bone_marrow.utilities import get_results_comparison_table

def get_stats_for_exp(outputs_repo: str):
    stats_filepath = os.path.join(outputs_repo, 'stats_latest-rank0.pth')
    if not os.path.isfile(stats_filepath):
        stats_filepath = os.path.join(outputs_repo, 'stats_latest.pth')
        
    stats = torch.load(stats_filepath, map_location='cpu')
    return stats

In [2]:
warehouse_repo = '.../warehouse/marrovision/bone_marrow_cell_classification/'

In [5]:
exp_warehouse_repo = {
    'resnext50_32x4d_imagenetpretrained': os.path.join(warehouse_repo, 'resnext50_32x4d_imagenetpretrained'),
    'resnext50_32x4d_mixup_100ep': [
        os.path.join(warehouse_repo, 'resnext50_32x4d_mixup_100ep', f'fold_{i}') for i in range(5)
    ]
}

In [6]:
exp_stats = dict()
for exp_name, filepath in exp_warehouse_repo.items():
    if isinstance(filepath, list):
        exp_stats[exp_name] = [get_stats_for_exp(e) for e in filepath]
    else:
        exp_stats[exp_name] = get_stats_for_exp(filepath)

In [12]:
mname = 'resnext50_32x4d_mixup_100ep'
df2 = []
for fold_index, stats in enumerate(exp_stats[mname]):
    tmp_df = get_results_comparison_table(stats['test']).sort_values(by='class_abbreviation')
    tmp_df = tmp_df.rename({e: mname + '_' + e[len('model_'):] for e in tmp_df.columns if e.startswith('model_')}, axis=1, errors='raise')
    tmp_df['fold_index'] = fold_index
    df2.append(tmp_df.copy())

df2 = pandas.concat(df2)

In [13]:
df2.groupby(['class_name', 'class_abbreviation', 'baseline_precision_strict',
       'baseline_recall_strict', 'baseline_support', 'baseline_f1_strict',]).mean().reset_index().sort_values(by='class_abbreviation')

,class_name,class_abbreviation,baseline_precision_strict,baseline_recall_strict,baseline_support,baseline_f1_strict,resnext50_32x4d_mixup_100ep_precision,resnext50_32x4d_mixup_100ep_recall,resnext50_32x4d_mixup_100ep_f1,test_support,f1_difference_from_mateketal,fold_index
0,Abnormal eosinophils,ABE,0.02,0.20,8,0.036364,0.200000,0.100000,0.133333,1.6,0.096970,2.0
1,Artefacts,ART,0.82,0.74,19630,0.777949,0.908621,0.837290,0.871399,3926.0,0.093450,2.0
3,Basophils,BAS,0.14,0.64,441,0.229744,0.534897,0.632610,0.577319,88.2,0.347575,2.0
4,Blasts,BLA,0.75,0.65,11973,0.696429,0.818138,0.819759,0.818916,2394.6,0.122488,2.0
6,Erythroblasts,EBO,0.88,0.82,27395,0.848941,0.949156,0.925899,0.937379,5479.0,0.088438,2.0
5,Eosinophils,EOS,0.85,0.91,5883,0.878977,0.952336,0.963624,0.957914,1176.6,0.078937,2.0
7,Faggot cells,FGC,0.17,0.63,47,0.267750,0.226648,0.293333,0.253047,9.4,-0.014703,2.0
8,Hairy cells,HAC,0.35,0.80,409,0.486957,0.632030,0.780157,0.697659,81.8,0.210702,2.0
20,Smudge cells,KSC,0.28,0.90,42,0.427119,0.657584,0.886111,0.743494,8.4,0.316375,2.0
9,Immature lymphocytes,LYI,0.08,0.53,65,0.139016,0.385839,0.353846,0.362839,13.0,0.223823,2.0


In [14]:
df2.groupby(['class_name', 'class_abbreviation', 'baseline_precision_strict',
       'baseline_recall_strict', 'baseline_support', 'baseline_f1_strict',]).std().reset_index().sort_values(by='class_abbreviation')

,class_name,class_abbreviation,baseline_precision_strict,baseline_recall_strict,baseline_support,baseline_f1_strict,resnext50_32x4d_mixup_100ep_precision,resnext50_32x4d_mixup_100ep_recall,resnext50_32x4d_mixup_100ep_f1,test_support,f1_difference_from_mateketal,fold_index
0,Abnormal eosinophils,ABE,0.02,0.20,8,0.036364,0.447214,0.223607,0.298142,0.547723,0.298142,1.581139
1,Artefacts,ART,0.82,0.74,19630,0.777949,0.009019,0.013228,0.005116,0.000000,0.005116,1.581139
3,Basophils,BAS,0.14,0.64,441,0.229744,0.068042,0.045551,0.044373,0.447214,0.044373,1.581139
4,Blasts,BLA,0.75,0.65,11973,0.696429,0.002968,0.010203,0.004903,0.547723,0.004903,1.581139
6,Erythroblasts,EBO,0.88,0.82,27395,0.848941,0.003675,0.003027,0.002667,0.000000,0.002667,1.581139
5,Eosinophils,EOS,0.85,0.91,5883,0.878977,0.005619,0.009437,0.004533,0.547723,0.004533,1.581139
7,Faggot cells,FGC,0.17,0.63,47,0.267750,0.103524,0.143716,0.121302,0.547723,0.121302,1.581139
8,Hairy cells,HAC,0.35,0.80,409,0.486957,0.051596,0.057883,0.048854,0.447214,0.048854,1.581139
20,Smudge cells,KSC,0.28,0.90,42,0.427119,0.131734,0.111285,0.066438,0.547723,0.066438,1.581139
9,Immature lymphocytes,LYI,0.08,0.53,65,0.139016,0.213995,0.258581,0.226117,0.000000,0.226117,1.581139
